This program executes one simulation using initial parameters that were set in 'main_code.ipynb' and are saved and imported from 'initial_parameters.py'.

# Import libraries

In [1]:
#import hoomd that ist saved on computer
import sys
sys.path.append("/net/theorie/home/niklas.butkevich/miniconda2/envs/py3/lib/python3.7/site-packages")
import hoomd
import hoomd.md

import numpy as np
import math
import os

from importlib import reload

#import file with initial parameters
import initial_parameters

#in general, python loads libraries only once
#if the content is changes (for example parameters) -> reload
initial_parameters = reload(initial_parameters)
path = ''

# Initialize the system

In [2]:
hoomd.context.initialize("")

HOOMD-blue 2.6.0 DOUBLE HPMC_MIXED TBB SSE SSE2 SSE3 
Compiled: 05/29/2019
Copyright (c) 2009-2019 The Regents of the University of Michigan.
-----
You are using HOOMD-blue. Please cite the following:
* J A Anderson, C D Lorenz, and A Travesset. "General purpose molecular dynamics
  simulations fully implemented on graphics processing units", Journal of
  Computational Physics 227 (2008) 5342--5359
* J Glaser, T D Nguyen, J A Anderson, P Liu, F Spiga, J A Millan, D C Morse, and
  S C Glotzer. "Strong scaling of general-purpose molecular dynamics simulations
  on GPUs", Computer Physics Communications 192 (2015) 97--107
-----
HOOMD-blue is running on the CPU


# Define the parameters (that were set in 'main_code.ipynb')

In [3]:
#define number of polymers segments (monomers)
Monomers = initial_parameters.Monomers

#define number of bonds between segments
Bonds = initial_parameters.Bonds

#set simulation box length
simulation_box_length = initial_parameters.simulation_box_length

#set box factor (factor to multiply each box side with)
box_factor = initial_parameters.box_factor

#define spring constant (Rouse model) for bonds between segments in units energy/distance^2
k = initial_parameters.k

#define rest bond length (same as LJ-sigma_AA)
r0 = initial_parameters.r0

#define temperature in units of kT
kT = initial_parameters.kT

#define drag-coefficient constant gamma for all particles
gamma = initial_parameters.gamma

#define translational diffusion constant
D_t = initial_parameters.D_t

#######################################################################

#set radius of passive particles
R_passive_partilce = initial_parameters.R_passive_partilce

#set volume of passive particles
volume_passive_particle = initial_parameters.volume_passive_particle

#set density of passive particles
density_passive_particles = initial_parameters.density_passive_particles

#set number of passive particles
n_passive_particles = initial_parameters.n_passive_particles

#######################################################################

#set radius of active particles (should be halb sigma for LJ-potential)
R_active_partilce = initial_parameters.R_active_partilce

#set volume of active particles
volume_active_particle = initial_parameters.volume_active_particle

#set density of active particles
density_active_particles = initial_parameters.density_active_particles

#set number of active particles
n_active_particles = initial_parameters.n_active_particles

#set active force vector for active particles
force_vec_active_particle = initial_parameters.force_vec_active_particle

#define velocity of active particles
v_ac = initial_parameters.v_ac

#set rotational diffusion constant D_r for active particles
D_r_active_particle = initial_parameters.D_r_active_particle

#define tau_r (persistence time) for active particles
tau_r_active_particle = initial_parameters.tau_r_active_particle

#######################################################################

#set epsilon and sigma for LJ-potential between all particles
#A=Monomers, B=passive crowders, C=active crowders
lj_epsilon_AA = initial_parameters.lj_epsilon_AA
lj_sigma_AA   = initial_parameters.lj_sigma_AA

lj_epsilon_AB = initial_parameters.lj_epsilon_AB
lj_sigma_AB   = initial_parameters.lj_sigma_AB

lj_epsilon_AC = initial_parameters.lj_epsilon_AC
lj_sigma_AC   = initial_parameters.lj_sigma_AC

lj_epsilon_BB = initial_parameters.lj_epsilon_BB
lj_sigma_BB   = initial_parameters.lj_sigma_BB

lj_epsilon_BC = initial_parameters.lj_epsilon_BC
lj_sigma_BC   = initial_parameters.lj_sigma_BC

lj_epsilon_CC = initial_parameters.lj_epsilon_CC
lj_sigma_CC   = initial_parameters.lj_sigma_CC

#define r_cut from which LJ-potential is 0 (same for all particles)
r_cut = initial_parameters.r_cut
#######################################################################

#define integration step time
dt = initial_parameters.dt

#define number of integration steps
integration_steps = initial_parameters.integration_steps

#define number of points for representation
number_of_points = initial_parameters.number_of_points

#set return period to log data to file
return_period = initial_parameters.return_period

# Create box with particles

In [4]:
#polymer is valid for Monomers >= 2
if Monomers < 2:
    Monomers = 0

#define box dimensions
Lx = box_factor * simulation_box_length
Ly = box_factor * simulation_box_length
Lz = box_factor * simulation_box_length

#define the system as snapshot, consider different types of particles
snapshot = hoomd.data.make_snapshot(Monomers+n_passive_particles+n_active_particles,
                                    
                                    #box-dimension: x-direction: -Lx/2 to Lx/2 etc.
                                    box=hoomd.data.boxdim(Lx=Lx, Ly=Ly, Lz=Lz),
                                    
                                    #list of types; typeid[0]='A', typeid[1]='B' etc.
                                    #A=Monomers, B=passive crowders, C=active crowders
                                    particle_types=['A','B','C'],
                                    bond_types=['polymer'])

# Set ID's, positions (and bonds)

In [5]:
#create list to set monomer's/particle's initial type-IDs, positions, bonds
initial_IDs = []

#make empty list for initial positions of monomers
initial_positions_monomers = []

#make empty list for initial positions of passive particles
initial_positions_passive_particles = []

#make empty list for initial positions of active particles
initial_positions_active_particles = []

#make empty list for bond between monomers
initial_bonds  =[]

#######################################################################################
#define initial quantities for polymer ONLY if Monomers >= 2
if Monomers >= 2:
    
    initial_position_of_cm_x = 0
    initial_position_of_cm_y = 0
    initial_position_of_cm_z = 0

    #set initial monomer positions and IDs; ATTENTION: particle with ID=0 is monomer,
    #last monomer has ID Monomers-1; paricles with ID >= Monomers are not part of chain
    for monomer in range(Monomers):
        initial_IDs.append(0)
        initial_positions_monomers.append([monomer - (Monomers/2) + 0.5, +Ly/2-1, +Lz/2-1])

        #calculate initial distance of center of mass from origin
        initial_position_of_cm_x += (initial_positions_monomers[monomer][0] / Monomers)
        initial_position_of_cm_y += (initial_positions_monomers[monomer][1] / Monomers)
        initial_position_of_cm_z += (initial_positions_monomers[monomer][2] / Monomers)

    initial_position_of_cm = [initial_position_of_cm_x,
                              initial_position_of_cm_y,
                              initial_position_of_cm_z]

    initial_end_to_end_vector = [(initial_positions_monomers[Monomers-1][0]-
                                  initial_positions_monomers[0][0]),
                                 (initial_positions_monomers[Monomers-1][1]-
                                  initial_positions_monomers[0][1]),
                                 (initial_positions_monomers[Monomers-1][2]-
                                  initial_positions_monomers[0][2])]

    #define the bonds
    for bond in range(Monomers-1):
        initial_bonds.append([bond, bond+1])

    #resize the bonds list to actual number of bonds
    snapshot.bonds.resize(Monomers-1)
#######################################################################################
#set initial passive particle positions and IDs, break loops if number of passive 
#particles is reached
passive_particles_counter = 0
for Lz_counter in range(-int(Lz/2), int(Lz/2)):
    
    if passive_particles_counter == n_passive_particles:
                break
            
    for Ly_counter in range(-int(Ly/2), int(Ly/2)):
        
        if passive_particles_counter == n_passive_particles:
                break  
                
        for Lx_counter in range(-int(Lx/2), int(Lx/2)):
            passive_particles_counter += 1
            #ID has to be the same as index-position in list
            #when define the snapshot above, so here: C-->1
            initial_IDs.append(1)
            initial_positions_passive_particles.append([Lx_counter, Ly_counter, Lz_counter])
    
            if passive_particles_counter == n_passive_particles:
                break
#######################################################################################
#set initial active particle positions and IDs, break loops if number of active
#particles is reached

"""ATTENTION: passive and active particles are set on a grid.
Therefore, go to next grid point if there is already a passive
particles on it."""
#use counter to check
check_pc_counter = 0

#use loop to set active particles
active_particles_counter = 0
for Lz_counter in range(-int(Lz/2), int(Lz/2)):
    
    if active_particles_counter == n_active_particles:
                break
            
    for Ly_counter in range(-int(Ly/2), int(Ly/2)):
        
        if active_particles_counter == n_active_particles:
                break  
                
        for Lx_counter in range(-int(Lx/2), int(Lx/2)):
            
            ##############################################################
            """ATTENTION: passive and active particles are set on a grid.
            Therefore, go to next grid point if there is already a passive
            particles on it."""
            if check_pc_counter < n_passive_particles:
                check_pc_counter += 1
                #if passive particle on the grid point, go to next point
                continue
            ##############################################################
            
            active_particles_counter += 1
            #ID has to be the same as index-position in list
            #when define the snapshot above, so here: C-->1
            initial_IDs.append(2)

            initial_positions_active_particles.append([Lx_counter, Ly_counter, Lz_counter])
    
            if active_particles_counter == n_active_particles:
                break
                
#copy IDs, positions and bonds into valid arrays
snapshot.particles.typeid[:] = initial_IDs
snapshot.particles.position[:] = (initial_positions_monomers+
                                  initial_positions_passive_particles+
                                  initial_positions_active_particles)
#set bonds
if Monomers >= 2:
    snapshot.bonds.group[:] = initial_bonds

In [6]:
print (snapshot.particles.typeid)
print (snapshot.particles.position)

[0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]
[[-4.5  4.   4. ]
 [-3.5  4.   4. ]
 [-2.5  4.   4. ]
 ...
 [ 2.   2.  -2. ]
 [ 3.   2.  -2. ]
 [ 4.   2.  -2. ]]


# Initialization of integration parameters

In [7]:
#initialize HOOMD using snapshot
system = hoomd.init.read_snapshot(snapshot)

#set the bond type and strength between monomers
harmonic = hoomd.md.bond.harmonic()
harmonic.bond_coeff.set('polymer', k=k, r0=r0)

#consider every particle
all = hoomd.group.all()

#make gsd file for simulation
hoomd.dump.gsd(path+'Monomers'+str(Monomers)+
                    '__k'+str(k)+
                    '__gamma'+str(gamma)+
                    '__kT'+str(kT)+
                    '__density_pc'+str(density_passive_particles)+
                    '__density_ac'+str(density_active_particles)+
                    '__Dr'+str(D_r_active_particle)+
                    '__v_ac'+str(v_ac)+
                    '.gsd',

                   #return_period = number of points in plots
                   period=return_period,
                   group=all, overwrite=True)

notice(2): Group "all" created containing 390 particles


# Set LJ potentials

In [8]:
#cell method is used if r_cut is for all pairs of particles the same
nl = hoomd.md.nlist.cell()

#LJ-potential depens on r_cut
lj = hoomd.md.pair.lj(r_cut=r_cut, nlist=nl)

lj.pair_coeff.set('A', 'A', epsilon=lj_epsilon_AA, sigma=lj_sigma_AA)
lj.pair_coeff.set('A', 'B', epsilon=lj_epsilon_AB, sigma=lj_sigma_AB)
lj.pair_coeff.set('A', 'C', epsilon=lj_epsilon_AC, sigma=lj_sigma_AC)
lj.pair_coeff.set('B', 'B', epsilon=lj_epsilon_BB, sigma=lj_sigma_BB)
lj.pair_coeff.set('B', 'C', epsilon=lj_epsilon_BC, sigma=lj_sigma_BC)
lj.pair_coeff.set('C', 'C', epsilon=lj_epsilon_CC, sigma=lj_sigma_CC)

# Set force vector for active particles

In [9]:
#set force vector for active particles (group 'C') only if
#there are active particles

if n_active_particles != 0:
    groupC = hoomd.group.type(name="C", type='C')

    #set active force vector list f_lst (equal for all active particles);
    #it is used in 'hoomd.md.force.active' below
    f_lst = []

    for active_paricle in range(n_active_particles):
        f_lst.append(force_vec_active_particle)


    hoomd.md.force.active(seed=np.random.randint(0,9999),

                          #consider group C (active particles)
                          group=groupC,

                          #set active force vector
                          f_lst=f_lst,

                          #set active torque vector
                          #t_lst=t_lst,

                          #if True: consider inertial system of particle
                          orientation_link=False,

                          #here False! (orientation does not match with
                          #particles active force vector)
                          orientation_reverse_link=False, 

                          #set rotational diffusion constant for ac
                          rotation_diff=D_r_active_particle,

                          #constraints are not considered
                          constraint=None)

notice(2): Group "C" created containing 190 particles


# Set integrator

In [10]:
#integrator
hoomd.md.integrate.mode_standard(dt=dt)
integrator = hoomd.md.integrate.brownian(group=all, kT=kT, dscale=False,
                                         seed=np.random.randint(0,9999))


#set gamma (friction constant); ATTENTION: set after integrator is set!
integrator.set_gamma(all, gamma=gamma)

notice(2): integrate.langevin/bd is using specified gamma values


# IMPORTANT FOR ANALYSIS OF PARTICLE POSITION

Using classes below, the MSD of the crowders as well as the squared end-to-end distance, MSD and the auto-correlation of the end-to-end vector of the polymer is analyzed. Positions of ALL particles (monomers, passive crowders and active crowders) can be taken from one array: 'snapshot.particles.position'. It is important to consider the correct index. snapshot.particles.position = initial_position_monomers (+ initial_positions_passive_particles) + initial_position_active_particles. For example, the index of the second crowder is then: Monomers+1.

For polymer: List of monomers is at the first position. 4th monomer has the index 4-1.

In general: particles_list = monomers_list + passive_particles_list + active_particles_list

# Polymer: Sq_end_to_end_distance

In [11]:
#create class for end-to-end distance
class Sq_end_to_end_distance:
    
    def __init__(self, system):
        
        self.system = system
        
    def __call__(self, timestep):
        
        #################################################################
        #(If there is no polymer in system, return end-to-end-distance=0)
        if Monomers == 0:
            current_sq_end_to_end_distance = 0
            return (current_sq_end_to_end_distance)
        #################################################################
        
        snapshot = self.system.take_snapshot()
        
        #ATTENTION: paricles.position[...] can also be position of passive particle!!!
        #paricles.position[0] (position 0) --> first monomer
        
        #find actual simulation box (if particle outside the box, it is in the next box), m=monomer
        m_first_image = snapshot.particles.image[0]
        m_last_image = snapshot.particles.image[Monomers-1]
        
        #calculate positions of first and last monomer Lx,Ly,Lz = length of simulation box
        m_first_x = snapshot.particles.position[0][0] + Lx*m_first_image[0]
        m_first_y = snapshot.particles.position[0][1] + Ly*m_first_image[1]
        m_first_z = snapshot.particles.position[0][2] + Lz*m_first_image[2]
        m_first = [m_first_x, m_first_y, m_first_z]
        
        m_last_x = snapshot.particles.position[Monomers-1][0] + Lx*m_last_image[0]
        m_last_y = snapshot.particles.position[Monomers-1][1] + Ly*m_last_image[1]
        m_last_z = snapshot.particles.position[Monomers-1][2] + Lz*m_last_image[2]  
        m_last = [m_last_x, m_last_y, m_last_z]
        
        #calculate squared end-to-end distance
        current_sq_end_to_end_distance = ((m_last[0]-m_first[0])**2 +
                                          (m_last[1]-m_first[1])**2 +
                                          (m_last[2]-m_first[2])**2)
        
        #return current end-to-end distance
        return (current_sq_end_to_end_distance)


# Polymer: MSD

In [12]:
#create class for distance of centre of mass from origin
class Sq_distance_of_cm:
    
    def __init__(self, system):
        
        self.system = system
        
    def __call__(self, timestep):
        
        #################################################
        #(If there is no polymer in system, return MSD=0)
        if Monomers == 0:
            current_sq_distance_of_cm = 0
            return (current_sq_distance_of_cm)
        #################################################
        
        snapshot = self.system.take_snapshot()
        
        #define lists of x,y and z positions of monomers
        list_of_x_positions = []
        list_of_y_positions = []
        list_of_z_positions = []
        
        #compute current position of centre of mass (the average is done by other Python-program)
        for monomers in range(Monomers):
            
            #Particles can move outside the box (and appear in the same box on the
            #other side). Therefore, the actual position has
            #to be recalculated by unsing the box-length Lx, Ly, Lz
            list_of_x_positions.append(snapshot.particles.position[monomers][0]
                                      + Lx*snapshot.particles.image[monomers][0])
            list_of_y_positions.append(snapshot.particles.position[monomers][1]
                                      + Ly*snapshot.particles.image[monomers][1])
            list_of_z_positions.append(snapshot.particles.position[monomers][2]
                                      + Lz*snapshot.particles.image[monomers][2])
            
        current_position_of_cm = [np.mean(list_of_x_positions),
                                  np.mean(list_of_y_positions),
                                  np.mean(list_of_z_positions)]
        
        #calculation of squared distance of cm from origin
        current_sq_distance_of_cm = ((current_position_of_cm[0]-initial_position_of_cm[0])**2 +
                                     (current_position_of_cm[1]-initial_position_of_cm[1])**2 +
                                     (current_position_of_cm[2]-initial_position_of_cm[2])**2)
        
        #return current end-to-end distance        
        return (current_sq_distance_of_cm)

# Polymer: Auto-correlation of end-to-end vector

In [13]:
#create class for auto correlation of end-to-end distance vector
class Auto_corr_ee_vector:
    
    def __init__(self, system):
        
        self.system = system
        
    def __call__(self, timestep):
        
        #################################################################
        #(If there is no polymer in system, return auto_corr_ee_vector=0)
        if Monomers == 0:
            auto_corr_ee_vector = 0
            return (auto_corr_ee_vector)
        #################################################################
        
        snapshot = self.system.take_snapshot()
        
        #ATTENTION: paricles.position[...] can also be position of passive particle!!!
        #paricles.position[0] (position 0) --> first monomer 
        
        current_end_to_end_vector = [(snapshot.particles.position[Monomers-1][0]
                                          + Lx*snapshot.particles.image[Monomers-1][0]) -
                                         
                                          (snapshot.particles.position[0][0]
                                          + Lx*snapshot.particles.image[0][0]),
                                        
                                    
                                        (snapshot.particles.position[Monomers-1][1]
                                          + Ly*snapshot.particles.image[Monomers-1][1]) -
                                         
                                          (snapshot.particles.position[0][1]
                                          + Ly*snapshot.particles.image[0][1]),
                                  
                                        
                                        (snapshot.particles.position[Monomers-1][2]
                                          + Lz*snapshot.particles.image[Monomers-1][2]) -
                                         
                                          (snapshot.particles.position[0][2]
                                          + Lz*snapshot.particles.image[0][2])]
        
        auto_corr_ee_vector = (current_end_to_end_vector[0] * initial_end_to_end_vector[0] +
                               current_end_to_end_vector[1] * initial_end_to_end_vector[1] +
                               current_end_to_end_vector[2] * initial_end_to_end_vector[2])
        
        #return current auto-corr end-to-end vector        
        return (auto_corr_ee_vector)

# MSD passive partilces

In [14]:
#create class for MSD of passive crowders
class Averaged_msd_of_pc:
    
    def __init__(self, system):
        
        self.system = system
        
    def __call__(self, timestep):
        
        ############################################################
        #(If there are no passive particles in system, return MSD=0)
        if n_passive_particles == 0:
            current_averaged_msd_pc = 0
            return (current_averaged_msd_pc)
        ############################################################
        
        snapshot = self.system.take_snapshot()

        #square distance of the center of mass of all passive particles (crowders)
        #is Mean Squared Displacement (msd); passive crowders = pc
        list_of_current_msd_of_all_pc = []
        
        for pc_index in range(n_passive_particles):
        
            #set the msd of current (new) particle to 0
            msd_current_pc = 0
        
            for axis in range(3): #axis=dimension; we simulate in 3D
        
                #square distance of the center of mass of all passive particles
                #is Mean Squared Displacement (msd)

                                           #LIST IS FOR ALL PARTICLES
                msd_current_pc += ( ((snapshot.particles.position[Monomers+pc_index][axis]
                                           #Lx=length of one axis, eqaul to other lengths (!)
                                           + Lx*snapshot.particles.image[Monomers+pc_index][axis]) -
                                         
                                           #ATTENTION: list is for pc, NOT FOR ALL PARTICLES
                                           initial_positions_passive_particles[pc_index][axis]) **2)
            
            
            #add the msd of one passive particle (pc) to list
            list_of_current_msd_of_all_pc.append(msd_current_pc)
            
        #average the list of msd of all particles
        current_averaged_msd_pc = np.mean(list_of_current_msd_of_all_pc)
        
        #return current average msd  
        return (current_averaged_msd_pc)

# MSD active particles

In [15]:
#create class for MSD of active crowders
class Averaged_msd_of_ac: 
    
    def __init__(self, system):
        
        self.system = system
        
    def __call__(self, timestep):
        
        ###########################################################
        #(If there are no active particles in system, return MSD=0)
        if n_active_particles == 0:
            current_averaged_msd_ac = 0
            return (current_averaged_msd_ac)
        ###########################################################
        
        snapshot = self.system.take_snapshot()

        #square distance of the center of mass of all active particles (crowders)
        #is Mean Squared Displacement (msd); active crowders = ac
        list_of_current_msd_of_all_ac = []
        
        for ac_index in range(n_active_particles):
        
            #set the msd of current (new) particle to 0
            msd_current_ac = 0
        
            for axis in range(3): #axis=dimension; we simulate in 3D
        
                #square distance of the center of mass of an active particles
                #is Mean Squared Displacement (msd)

                                           #LIST IS FOR ALL PARTICLES
                msd_current_ac += ( ((snapshot.particles.position[Monomers+n_passive_particles+ac_index][axis]
                                           #Lx=length of one axis, eqaul to other lengths (!)
                                           + Lx*snapshot.particles.image[Monomers+n_passive_particles+ac_index][axis]) -
                                         
                                           #ATTENTION: list is for ac, NOT FOR ALL PARTICLES
                                           initial_positions_active_particles[ac_index][axis]) **2)
            
            
            #add the msd of one acrive particle (ac) to list
            list_of_current_msd_of_all_ac.append(msd_current_ac)
            
        #average the list of msd of all particles
        current_averaged_msd_ac = np.mean(list_of_current_msd_of_all_ac)
        
        #return current average msd
        return (current_averaged_msd_ac)

# Create instances for classes and log the quantities

In [16]:
#for polymer
instance_sq_end_to_end_distance = Sq_end_to_end_distance(system)
instance_sq_distance_of_cm      = Sq_distance_of_cm(system)
instance_auto_corr_ee_vector    = Auto_corr_ee_vector(system)

#for crowders
instance_averaged_msd_of_pc = Averaged_msd_of_pc(system)
instance_averaged_msd_of_ac = Averaged_msd_of_ac(system)

#use analyze.log to write quantities of simulation into a text file
logger = hoomd.analyze.log(filename=('Monomers'+str(Monomers)+
                                     '__k'+str(k)+
                                     '__gamma'+str(gamma)+
                                     '__kT'+str(kT)+
                                     '__density_pc'+str(density_passive_particles)+
                                     '__density_ac'+str(density_active_particles)+
                                     '__Dr'+str(D_r_active_particle)+
                                     '__v_ac'+str(v_ac)+
                                     '.dat'),
                           quantities=['sq_end_to_end_distance',
                                       'sq_distance_of_cm',
                                       'auto_corr_ee_vector',
                                       'current_averaged_msd_pc',
                                       'current_averaged_msd_ac',],
                           
                           #return and write to file
                           period = return_period,
                           header_prefix='#',
                           overwrite=True)

#create a new quantity that is logged due to expression above to a text file
logger.register_callback(('sq_end_to_end_distance'), instance_sq_end_to_end_distance)
logger.register_callback(('sq_distance_of_cm'), instance_sq_distance_of_cm)
logger.register_callback(('auto_corr_ee_vector'), instance_auto_corr_ee_vector)

logger.register_callback(('current_averaged_msd_pc'), instance_averaged_msd_of_pc)
logger.register_callback(('current_averaged_msd_ac'), instance_averaged_msd_of_ac)

# Run the simulation

In [ ]:
hoomd.run(integration_steps)

notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 0 exclusions             : 380
notice(2): Particles with 1 exclusions             : 2
notice(2): Particles with 2 exclusions             : 8
notice(2): Neighbors included by diameter          : no
notice(2): Neighbors excluded when in the same body: no
** starting run **
Time 00:00:10 | Step 56760 / 10000000 | TPS 5675.96 | ETA 00:29:11
Time 00:00:20 | Step 116259 / 10000000 | TPS 5949.86 | ETA 00:27:41
Time 00:00:30 | Step 175837 / 10000000 | TPS 5957.77 | ETA 00:27:28
Time 00:00:40 | Step 235001 / 10000000 | TPS 5912.67 | ETA 00:27:31
Time 00:00:50 | Step 294069 / 10000000 | TPS 5906.77 | ETA 00:27:23
Time 00:01:00 | Step 353644 / 10000000 | TPS 5957.44 | ETA 00:26:59
Time 00:01:10 | Step 413078 / 10000000 | TPS 5943.39 | ETA 00:26:53
Time 00:01:20 | Step 472088 / 10000000 | TPS 5900.98 | ETA 00:26:54
Time 00:01:30 | Step 530565 / 10000000 | TPS 5847.68 | ETA 00:26:59
Time 00:01:40 | Step 589684 / 10000000

# Analysis of simulations

In [ ]:
#create lists for quantities for graphical representation (plots)
list_of_times = []

#polymer quantities lists
list_of_sq_end_to_end_distances = []
list_of_sq_distance_of_cm       = []
list_of_auto_corr_ee_vector     = []

#MSD lists for passive and active crowders
list_of_averaged_msd_pc = []
list_of_averaged_msd_ac = []

import csv

averaged_quantities = open('Monomers'+str(Monomers)+
                           '__k'+str(k)+
                           '__gamma'+str(gamma)+
                           '__kT'+str(kT)+
                           '__density_pc'+str(density_passive_particles)+
                           '__density_ac'+str(density_active_particles)+
                           '__Dr'+str(D_r_active_particle)+
                           '__v_ac'+str(v_ac)+
                           '.dat','r')
lines = csv.reader(averaged_quantities, delimiter='	')

#skip the header
next(lines)
    
for line in lines: #line content in general: time, sq_ee_distance, MSD_polymer.
                   #auto_corr_ee_vector, MSD_pc, MSD_ac

        #time has to be multiplied by dt, otherwise it is number of timesteps
        list_of_times.append(float(line[0])*initial_parameters.dt)
        
        #polymer quantities
        list_of_sq_end_to_end_distances.append(float(line[1]))
        list_of_sq_distance_of_cm.append(float(line[2]))
        list_of_auto_corr_ee_vector.append(float(line[3]))
        
        #MSD of passive and active crowders
        list_of_averaged_msd_pc.append(float(line[4]))
        list_of_averaged_msd_ac.append(float(line[5]))

#close the file
averaged_quantities.close()

In [ ]:
import matplotlib.pyplot as plt

#change figure size
figure_size = plt.rcParams["figure.figsize"]
figure_size[0] = 9
figure_size[1] = 6
plt.rcParams['figure.figsize'] = figure_size

#set line width
plt.rcParams['lines.linewidth'] = 4

#set label size in the plots
plt.rcParams.update({'font.size': 18})

# Analysis: polymer sq_ee_dinstance

In [ ]:
#plot squared end-to-end distance as function of time
#convert array into numpy array and use short names (np.array(np.array(list)) is still np.array)
t = np.array(list_of_times)
y = np.array(list_of_sq_end_to_end_distances)

#plot the results
plt.plot(t, y, 'b.', label='data')

#set y range due to use of log-representation (otherwise y axis starts at extremely small values)
plt.ylim(0, None)

#save the plot
plt.legend(loc='best')
plt.xlabel(r'$t$')
plt.ylabel(r'$< R_{\mathrm{ee}}^2(t) >$')
plt.draw()
plt.savefig(path+
            'polymer_R_ee'+
            '__Monomers'+str(Monomers)+
            '__k'+str(k)+
            '__gamma'+str(gamma)+
            '__kT'+str(kT)+
            '__density_pc'+str(density_passive_particles)+
            '__density_ac'+str(density_active_particles)+
            '__Dr'+str(D_r_active_particle)+
            '__v_ac'+str(v_ac)+
            '.png')
plt.show()
plt.close()

# Analysis: polymer MSD

In [ ]:
#MSD = sq_distance_of_cm

#covert arrays into numpy arrays and use short names (use t and y)
t = np.array(list_of_times)
y = np.array(list_of_sq_distance_of_cm)

#plot the results
plt.plot(t, y, 'b.', label='data')



#set y range due to use of log-representation (otherwise y axis starts at extremely small values)
plt.ylim(0.1, None)

#save the plot
plt.legend(loc='best')
plt.xscale('log')
plt.yscale('log')
plt.xlabel(r'$t$')
plt.ylabel(r'$< \sigma_{\mathrm{cm}}^2(t) >$')
plt.draw()
plt.savefig(path+
            'polymer_MSD'+
            '__Monomers'+str(Monomers)+
            '__k'+str(k)+
            '__gamma'+str(gamma)+
            '__kT'+str(kT)+
            '__density_pc'+str(density_passive_particles)+
            '__density_ac'+str(density_active_particles)+
            '__Dr'+str(D_r_active_particle)+
            '__v_ac'+str(v_ac)+
            '.png')
plt.show()
plt.close()

# Analysis: polymer Auto_corr_ee_vector

In [ ]:
#covert arrays into numpy arrays and use short names (full=all values)
t = np.array(list_of_times)
y = np.array(list_of_auto_corr_ee_vector)

#plot the results
plt.plot(t, y, 'b.', label='data')

#save plot
plt.legend(loc='best')
plt.xlabel(r'$t$')
plt.yscale('log')
plt.ylabel(r'$< \phi(t) >$')
plt.draw()
plt.savefig(path+
            'polymer_auto_corr_ee_vec'+
            '__Monomers'+str(Monomers)+
            '__k'+str(k)+
            '__gamma'+str(gamma)+
            '__kT'+str(kT)+
            '__density_pc'+str(density_passive_particles)+
            '__density_ac'+str(density_active_particles)+
            '__Dr'+str(D_r_active_particle)+
            '__v_ac'+str(v_ac)+
            '.png')
plt.show()
plt.close()

# MSD passive crowders

In [ ]:
#convert array into numpy array and use short names
t = np.array(list_of_times)
y = np.array(list_of_averaged_msd_pc)

plt.plot(t, y,
         'b.', label=('data'))

#define x-values to plot function (interval, step length)
x = np.arange(0., 10**3, 0.1)
plt.plot(x, (6*D_t*x),
             'r--',
             #set transparency of line
             alpha=0.8, label=('theory'))



#save the plot
plt.xscale('log')
plt.yscale('log')
plt.legend(loc='best')
plt.xlabel(r'$t$')
plt.ylabel(r'$< \sigma_{\mathrm{pc}}^2(t) >$')
plt.draw()
plt.savefig(path+
            'MSD_pc'+
            '__Monomers'+str(Monomers)+
            '__k'+str(k)+
            '__gamma'+str(gamma)+
            '__kT'+str(kT)+
            '__density_pc'+str(density_passive_particles)+
            '__density_ac'+str(density_active_particles)+
            '__Dr'+str(D_r_active_particle)+
            '__v_ac'+str(v_ac)+
            '.png')
plt.show()
plt.close()

# MSD active crowders

In [ ]:
#convert array into numpy array and use short names
t = np.array(list_of_times)
y = np.array(list_of_averaged_msd_ac)

plt.plot(t, y,
         'b.', label=('data'))

#plt.plot(t, 6*gamma*t, 'r--', label=('6*gamma*t'))
#plt.plot(t, t**2, 'g--', label=('(1**2)*(t**2)'))

#define x-values to plot function (interval, step length)
x = np.arange(0., 10**3, 0.1)
plt.plot(x, (6*D_t*x + 
             2*(v_ac**2)*tau_r_active_particle*x + 
             2*(v_ac**2)*(tau_r_active_particle**2)*(np.exp(-x/tau_r_active_particle) -1 )),
             'r--',
             #set transparency of line
             alpha=0.8, label=('theory'))



#save the plot
plt.xscale('log')
plt.yscale('log')
plt.legend(loc='best')
plt.xlabel(r'$t$')
plt.ylabel(r'$< \sigma_{\mathrm{ac}}^2(t) >$')
plt.draw()
plt.savefig(path+
            'MSD_ac'+
            '__Monomers'+str(Monomers)+
            '__k'+str(k)+
            '__gamma'+str(gamma)+
            '__kT'+str(kT)+
            '__density_pc'+str(density_passive_particles)+
            '__density_ac'+str(density_active_particles)+
            '__Dr'+str(D_r_active_particle)+
            '__v_ac'+str(v_ac)+
            '.png')
plt.show()
plt.close()